# Business Question:
## How do rising housing costs, as measured by home values, influence homelessness rates in San Francisco, and what interventions can mitigate this trend?
This study analyzes historical trends in home values and their correlation with homelessness rates in San Francisco. By identifying key affordability challenges, we aim to explore policy interventions, such as zoning reforms and affordable housing initiatives, to mitigate displacement and improve housing stability. Insights from this research will inform strategies for sustainable and inclusive urban development.

In [ ]:
#Libraries 
import pandas as pd
import numpy as np 
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Data Wrangling

In [ ]:
# Load the uploaded dataset to inspect its structure and contents
file_path = '/Users/diana/Babson MSBA/Advance Programming for BA/Phyton/raw original data/Metro_zhvi_uc_condo_tier_0.33_0.67_sm_sa_month.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head(), data.info()


In [ ]:
# Check for missing values in each column
missing_values = data.isnull().sum()

print("Missing values per column:")
print(missing_values)


# Identify columns that should be numeric but are stored as objects
for col in data.columns:
    if data[col].dtype == 'object':  # Checking text-based columns
        try:
            data[col].astype(float)  # Attempt conversion
            print(f"⚠️ Column '{col}' should be numeric but is stored as object.")
        except ValueError:
            pass  # Column contains non-numeric values, so it's fine as object

# List numeric columns
numeric_cols = data.select_dtypes(include=['int64', 'float64']).columns

# Find non-numeric values in numeric columns
for col in numeric_cols:
    invalid_values = data[~data[col].astype(str).str.replace('.', '', 1).str.isnumeric()]
    if not invalid_values.empty:
        print(f"🚨 Non-numeric values found in '{col}':")
        print(invalid_values[[col]])
# Identify potential date columns
for col in data.columns:
    try:
        pd.to_datetime(data[col])
        print(f"📅 Column '{col}' looks like a date but is stored as '{data[col].dtype}'.")
    except Exception:
        pass  # Not a date column
# Convert a column to numeric
data["column_name"] = pd.to_numeric(data["column_name"], errors='coerce')

# Convert a column to datetime
data["date_column"] = pd.to_datetime(data["date_column"], errors='coerce')



In [ ]:
# Identify metadata columns (non-date columns)
metadata_cols = ['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName']

# Identify actual date columns (all other columns are dates)
date_cols = [col for col in data.columns if col not in metadata_cols]

# Reshape the dataset into a long format for time-series analysis
data_melted = data.melt(id_vars=metadata_cols, 
                         var_name="Date", 
                         value_name="HomeValue")

# Convert Date column to datetime format
data_melted["Date"] = pd.to_datetime(data_melted["Date"])

# Display the first few rows of the reshaped dataset
data_melted.head()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set figure size for better visualization
plt.figure(figsize=(12, 6))

# Create a box plot to detect outliers in home values
sns.boxplot(x=data_melted["HomeValue"])

# Add title and labels
plt.title("Box Plot of Home Values", fontsize=14)
plt.xlabel("Home Value", fontsize=12)

# Show the plot
plt.show()


# San Francisco, CA

In [ ]:
# Filter the dataset for San Francisco
sf_data = data_melted[data_melted["RegionName"] == "San Francisco, CA"]


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot home value trends in San Francisco
plt.figure(figsize=(12, 6))
sns.lineplot(data=sf_data, x="Date", y="HomeValue", label="Home Value", color="blue")
plt.title("San Francisco Home Value Trends (2000-2024)", fontsize=14)
plt.xlabel("Year", fontsize=12)
plt.ylabel("Home Value ($)", fontsize=12)
plt.legend()
plt.show()


In [ ]:
print(data_melted.head())  # This should display the first few rows
print(data_melted.info())  # This should confirm columns and data types


In [ ]:
print(data_melted["RegionName"].unique())  # Check available region names


In [ ]:
sf_data = data_melted[data_melted["RegionName"] == "San Francisco, CA"]
ca_data = data_melted[data_melted["StateName"] == "CA"]  # Get all California regions

# Aggregate California's home values by averaging all metro areas in the state
ca_avg = ca_data.groupby("Date")["HomeValue"].mean().reset_index()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))

# Plot San Francisco
sns.lineplot(data=sf_data, x="Date", y="HomeValue", label="San Francisco, CA", color="blue", linewidth=2)

# Plot California State (Averaged)
sns.lineplot(data=ca_avg, x="Date", y="HomeValue", label="California State (Average)", color="red", linewidth=2)

# Add title and labels
plt.title("Comparison of Home Value Trends: San Francisco vs. California", fontsize=14)
plt.xlabel("Year", fontsize=12)
plt.ylabel("Home Value ($)", fontsize=12)
plt.legend()

# Show the plot
plt.show()


In [ ]:
file_path = '/Users/diana/Babson MSBA/Advance Programming for BA/Phyton/raw original data/Metro_zhvi_uc_condo_tier_0.33_0.67_sm_sa_month.csv'

In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# Load the dataset
file_path = '/Users/diana/Babson MSBA/Advance Programming for BA/Phyton/raw original data/Metro_zhvi_uc_condo_tier_0.33_0.67_sm_sa_month.csv'
data = pd.read_csv(file_path)

# Identify metadata columns
metadata_cols = ['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName']

# Identify actual date columns (all columns except metadata)
date_cols = [col for col in data.columns if col not in metadata_cols]

# Reshape the dataset into long format
data_melted = data.melt(id_vars=metadata_cols, var_name="Date", value_name="HomeValue")

# Convert Date column to datetime format
data_melted["Date"] = pd.to_datetime(data_melted["Date"], errors='coerce')

# Filter for San Francisco
sf_data = data_melted[data_melted["RegionName"] == "San Francisco, CA"].copy()

# Set Date as index and keep only HomeValue
sf_data = sf_data.set_index("Date")[["HomeValue"]].dropna()

# Train the ARIMA model
arima_model = ARIMA(sf_data, order=(2, 1, 2))  # Adjust order if needed
arima_result = arima_model.fit()

# Forecast for the next 12 months
forecast = arima_result.forecast(steps=12)

# Create future dates for visualization
future_dates = pd.date_range(start=sf_data.index[-1], periods=12, freq='M')

# Plot actual vs. forecasted values
plt.figure(figsize=(12, 6))
plt.plot(sf_data, label="Actual Home Values", color="blue")
plt.plot(future_dates, forecast, label="Forecast", color="red", linestyle="dashed")

# Add title and labels
plt.title("Housing Value Forecast for San Francisco (ARIMA Model)", fontsize=14)
plt.xlabel("Year", fontsize=12)
plt.ylabel("Home Value ($)", fontsize=12)
plt.legend()

# Show the plot
plt.show()
